# Implement Cycle GAN for medical images segmentation

In [32]:
import tensorflow as tf
import tf_slim
from collections import namedtuple

In [33]:
def instance_norm(input, name="instance_norm"):
    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        depth = input.get_shape()[3]
        scale = tf.compat.v1.get_variable("scale", [depth], initializer=tf.compat.v1.random_normal_initializer(1.0, 0.02, dtype=tf.float32))
        offset = tf.compat.v1.get_variable("offset", [depth], initializer=tf.compat.v1.constant_initializer(0.0))
        mean, variance = tf.compat.v1.nn.moments(input, axes=[1,2], keep_dims=True)
        epsilon = 1e-5
        inv = tf.compat.v1.rsqrt(variance + epsilon)
        normalized = (input-mean)*inv
        return scale*normalized + offset

In [34]:
def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak*x)

In [35]:
def conv2d(input_, output_dim, ks=4, s=2, stddev=0.02, padding='SAME', name="conv2d"):
    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        return tf_slim.conv2d(input_, output_dim, ks, s, padding=padding, activation_fn=None,
                            weights_initializer=tf.compat.v1.truncated_normal_initializer(stddev=stddev),
                            biases_initializer=None)

In [36]:
def discriminator(image, options, reuse=False, name="discriminator"):

    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        # image is 256 x 256 x input_c_dim
        if reuse:
            tf.compat.v1.get_variable_scope().reuse_variables()
        else:
            assert tf.compat.v1.get_variable_scope().reuse is False

        h0 = lrelu(conv2d(image, options.df_dim, name='d_h0_conv'))
        # h0 is (128 x 128 x self.df_dim)
        h1 = lrelu(instance_norm(conv2d(h0, options.df_dim*2, name='d_h1_conv'), 'd_bn1'))
        # h1 is (64 x 64 x self.df_dim*2)
        h2 = lrelu(instance_norm(conv2d(h1, options.df_dim*4, name='d_h2_conv'), 'd_bn2'))
        # h2 is (32x 32 x self.df_dim*4)
        h3 = lrelu(instance_norm(conv2d(h2, options.df_dim*8, s=1, name='d_h3_conv'), 'd_bn3'))
        # h3 is (32 x 32 x self.df_dim*8)
        h4 = conv2d(h3, 1, s=1, name='d_h3_pred')
        # h4 is (32 x 32 x 1)
        return h4

In [37]:
def mae_criterion(in_, target):
    return tf.reduce_mean((in_-target)**2)

In [38]:
class cyclegan(object):
    def __init__(self, sess):
        self.sess = sess
        self.batch_size = 1
        self.image_size = 256
        self.input_c_dim = 3
        self.output_c_dim = 3
        
        self.discriminator = discriminator
        self.criterionGAN = mae_criterion
        OPTIONS = namedtuple('OPTIONS', 'batch_size image_size \
                              gf_dim df_dim output_c_dim is_training')
        self.options = OPTIONS._make((self.batch_size, self.image_size,
                                      64, 64, self.output_c_dim,
                                      True))

        self._build_model()

In [39]:
class cyclegan(cyclegan):
    def _build_model(self):
        self.real_data = tf.compat.v1.placeholder(tf.float32,
                                        [None, self.image_size, self.image_size,
                                         self.input_c_dim + self.output_c_dim],
                                        name='real_A_and_B_images')
        self.real_A = self.real_data[:, :, :, :self.input_c_dim]
        self.real_B = self.real_data[:, :, :, self.input_c_dim:self.input_c_dim + self.output_c_dim]
        
        self.fake_A_sample = tf.compat.v1.placeholder(tf.float32,
                                            [None, self.image_size, self.image_size,
                                             self.input_c_dim], name='fake_A_sample')
        self.fake_B_sample = tf.compat.v1.placeholder(tf.float32,
                                            [None, self.image_size, self.image_size,
                                             self.output_c_dim], name='fake_B_sample')
        self.DB_real = self.discriminator(self.real_B, self.options, reuse=True, name="discriminatorB")
        self.DA_real = self.discriminator(self.real_A, self.options, reuse=True, name="discriminatorA")
        self.DB_fake_sample = self.discriminator(self.fake_B_sample, self.options, reuse=True, name="discriminatorB")
        self.DA_fake_sample = self.discriminator(self.fake_A_sample, self.options, reuse=True, name="discriminatorA")
        self.db_loss_real = self.criterionGAN(self.DB_real, tf.ones_like(self.DB_real))
        self.db_loss_fake = self.criterionGAN(self.DB_fake_sample, tf.zeros_like(self.DB_fake_sample))
        self.db_loss = (self.db_loss_real + self.db_loss_fake) / 2
        self.da_loss_real = self.criterionGAN(self.DA_real, tf.ones_like(self.DA_real))
        self.da_loss_fake = self.criterionGAN(self.DA_fake_sample, tf.zeros_like(self.DA_fake_sample))
        self.da_loss = (self.da_loss_real + self.da_loss_fake) / 2
        self.d_loss = self.da_loss + self.db_loss
        
        t_vars = tf.compat.v1.trainable_variables()
        print(t_vars)
        self.d_vars = [var for var in t_vars if 'discriminator' in var.name]


beta1 is momentum term of Adam

In [40]:
class cyclegan(cyclegan):
    def train(self):
        self.lr = tf.compat.v1.placeholder(tf.float32, None, name='learning_rate')
        self.d_optim = tf.compat.v1.train.AdamOptimizer(self.lr, beta1=0.5) \
        .minimize(self.d_loss, var_list=self.d_vars)
#         self.g_optim = tf.train.AdamOptimizer(self.lr, beta1=0.5) \
#             .minimize(self.g_loss, var_list=self.g_vars)
        


In [41]:
tfconfig = tf.compat.v1.ConfigProto(allow_soft_placement=True)
tfconfig.gpu_options.allow_growth = True
with tf.compat.v1.Session(config=tfconfig) as sess:
    model = cyclegan(sess)
    model.train()

[<tf.Variable 'discriminatorB/d_h0_conv/Conv/weights:0' shape=(4, 4, 3, 64) dtype=float32>, <tf.Variable 'discriminatorB/d_h1_conv/Conv/weights:0' shape=(4, 4, 64, 128) dtype=float32>, <tf.Variable 'discriminatorB/d_bn1/scale:0' shape=(128,) dtype=float32>, <tf.Variable 'discriminatorB/d_bn1/offset:0' shape=(128,) dtype=float32>, <tf.Variable 'discriminatorB/d_h2_conv/Conv/weights:0' shape=(4, 4, 128, 256) dtype=float32>, <tf.Variable 'discriminatorB/d_bn2/scale:0' shape=(256,) dtype=float32>, <tf.Variable 'discriminatorB/d_bn2/offset:0' shape=(256,) dtype=float32>, <tf.Variable 'discriminatorB/d_h3_conv/Conv/weights:0' shape=(4, 4, 256, 512) dtype=float32>, <tf.Variable 'discriminatorB/d_bn3/scale:0' shape=(512,) dtype=float32>, <tf.Variable 'discriminatorB/d_bn3/offset:0' shape=(512,) dtype=float32>, <tf.Variable 'discriminatorB/d_h3_pred/Conv/weights:0' shape=(4, 4, 512, 1) dtype=float32>, <tf.Variable 'discriminatorA/d_h0_conv/Conv/weights:0' shape=(4, 4, 3, 64) dtype=float32>, <tf

ValueError: Variable discriminatorB/d_h0_conv/Conv/weights/Adam/ already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope?